In [2]:
import pandas as pd
import numpy as np
import time

from tqdm.auto import tqdm
from basketball_reference_web_scraper.data import OutputType, OutputWriteOption
from basketball_reference_web_scraper.client import players_season_totals, players_advanced_season_totals

In [6]:
player_totals_basic = pd.DataFrame()
for i in range(2000, 2025):
    season_totals = pd.DataFrame(players_season_totals(season_end_year=i))
    season_totals['season_end'] = [i] * len(season_totals)
    player_totals_basic = pd.concat([player_totals_basic, season_totals], axis=0)
player_totals_basic.set_index('season_end', inplace=True)
player_totals_basic.to_csv('basics.csv')

KeyboardInterrupt: 

In [38]:
player_totals_basic = pd.read_csv('2000_2024_player_totals_basic.csv', index_col=0)
player_totals_basic

,slug,name,positions,age,team,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
season_end,,,,,,,,,,,,,,,,,,,,,
2000,abdulta01,Tariq Abdul-Wahad,[<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>],25,Team.ORLANDO_MAGIC,46,46,1205,223,515,...,115,151,77,162,72,53,16,87,116,563
2000,abdulta01,Tariq Abdul-Wahad,[<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>],25,Team.DENVER_NUGGETS,15,10,373,51,131,...,31,42,24,28,26,6,12,19,31,134
2000,abdursh01,Shareef Abdur-Rahim,[<Position.SMALL_FORWARD: 'SMALL FORWARD'>],23,Team.VANCOUVER_GRIZZLIES,82,82,3223,594,1277,...,446,551,218,607,271,89,87,249,244,1663
2000,alexaco01,Cory Alexander,[<Position.POINT_GUARD: 'POINT GUARD'>],26,Team.DENVER_NUGGETS,29,2,329,28,98,...,17,22,8,34,58,24,2,28,39,82
2000,allenra02,Ray Allen,[<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>],24,Team.MILWAUKEE_BUCKS,82,82,3070,642,1411,...,353,398,83,276,308,110,19,183,187,1809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,youngth01,Thaddeus Young,[<Position.POWER_FORWARD: 'POWER FORWARD'>],35,Team.TORONTO_RAPTORS,11,3,99,16,27,...,1,3,6,12,16,7,1,7,13,34
2024,youngtr01,Trae Young,[<Position.POINT_GUARD: 'POINT GUARD'>],25,Team.ATLANTA_HAWKS,36,36,1312,302,717,...,258,302,19,88,391,52,8,153,73,980
2024,yurtsom01,Omer Yurtseven,[<Position.CENTER: 'CENTER'>],25,Team.UTAH_JAZZ,23,6,254,38,76,...,6,10,35,66,13,4,10,22,33,85


In [39]:
player_totals_basic_cleaned = player_totals_basic.groupby('season_end')
agg_functions = {'slug': 'last', 'positions': 'last', 'age': 'last', 'team': 'last'}
agg_functions.update(dict(zip(player_totals_basic.columns[5:], ['sum'] * len(player_totals_basic.columns[5:]))))
player_totals_basic_cleaned = player_totals_basic_cleaned.transform(lambda x: x).groupby(['season_end', 'name']).aggregate(agg_functions)
player_totals_basic_cleaned['made_two_point_field_goals'] = player_totals_basic_cleaned['made_field_goals'] - player_totals_basic_cleaned['made_three_point_field_goals']
player_totals_basic_cleaned['attempted_two_point_field_goals'] = player_totals_basic_cleaned['attempted_field_goals'] - player_totals_basic_cleaned['attempted_three_point_field_goals']
player_totals_basic_cleaned

slug  \
season_end name                         
2000       A.C. Green       greenac01   
           A.J. Bramlett    bramlaj01   
           Aaron McKie      mckieaa01   
           Aaron Williams   williaa01   
           Adam Keefe       keefead01   
...                               ...   
2024       Zach Collins     colliza01   
           Zach LaVine      lavinza01   
           Zeke Nnaji       nnajize01   
           Ziaire Williams  willizi02   
           Zion Williamson  willizi01   

                                                                positions  \
season_end name                                                             
2000       A.C. Green         [<Position.POWER_FORWARD: 'POWER FORWARD'>]   
           A.J. Bramlett                    [<Position.CENTER: 'CENTER'>]   
           Aaron McKie      [<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>]   
           Aaron Williams     [<Position.POWER_FORWARD: 'POWER FORWARD'>]   
           Adam Keefe                       [<Position.CENTER: 'CENTER'>]   
...                                                                   ...   
2024       Zach Collins                     [<Position.CENTER: 'CENTER'>]   
           Zach LaVine      [<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>]   
           Zeke Nnaji         [<Position.POWER_FORWARD: 'POWER FORWARD'>]   
           Ziaire Williams    [<Position.SMALL_FORWARD: 'SMALL FORWARD'>]   
           Zion Williamson    [<Position.POWER_FORWARD: 'POWER FORWARD'>]   

                            age                       team  games_played  \
season_end name                                                            
2000       A.C. Green        36    Team.LOS_ANGELES_LAKERS            82   
           A.J. Bramlett     23   Team.CLEVELAND_CAVALIERS             8   
           Aaron McKie       27    Team.PHILADELPHIA_76ERS            82   
           Aaron Williams    28    Team.WASHINGTON_WIZARDS            81   
           Adam Keefe        29             Team.UTAH_JAZZ            62   
...                         ...                        ...           ...   
2024       Zach Collins      26     Team.SAN_ANTONIO_SPURS            31   
           Zach LaVine       28         Team.CHICAGO_BULLS            23   
           Zeke Nnaji        23        Team.DENVER_NUGGETS            34   
           Ziaire Williams   22     Team.MEMPHIS_GRIZZLIES            37   
           Zion Williamson   23  Team.NEW_ORLEANS_PELICANS            32   

                            games_started  minutes_played  made_field_goals  \
season_end name                                                               
2000       A.C. Green                  82            1929               173   
           A.J. Bramlett                0              61                 4   
           Aaron McKie                 14            1952               244   
           Aaron Williams               0            1545               235   
           Adam Keefe                   3             604                53   
...                                   ...             ...               ...   
2024       Zach Collins                23             810               153   
           Zach LaVine                 21             813               160   
           Zeke Nnaji                   0             328                40   
           Ziaire Williams              8             700               106   
           Zion Williamson             32             972               277   

                            attempted_field_goals  \
season_end name                                     
2000       A.C. Green                         387   
           A.J. Bramlett                       21   
           Aaron McKie                        593   
           Aaron Williams                     450   
           Adam Keefe                         130   
...                                           ...   
2024       Zach Collins                       309   
   

In [41]:
player_totals_basic_pg = pd.DataFrame()
games_played = player_totals_basic_cleaned['games_played']
player_totals_basic_pg['MP/G'] = player_totals_basic_cleaned['minutes_played'] / games_played
player_totals_basic_pg['GS%'] = player_totals_basic_cleaned['games_started'] / games_played
player_totals_basic_pg['PS/G'] = player_totals_basic_cleaned['points'] / games_played
player_totals_basic_pg['FG/G'] = player_totals_basic_cleaned['made_field_goals'] / games_played
player_totals_basic_pg['FGA'] = player_totals_basic_cleaned['attempted_field_goals'] / games_played
player_totals_basic_pg['FG%'] = player_totals_basic_cleaned['made_field_goals'] / player_totals_basic_cleaned['attempted_field_goals']
player_totals_basic_pg['2P/G'] = player_totals_basic_cleaned['made_two_point_field_goals'] / games_played
player_totals_basic_pg['2PA/G'] = player_totals_basic_cleaned['attempted_two_point_field_goals'] / games_played
player_totals_basic_pg['2P%'] = player_totals_basic_cleaned['made_two_point_field_goals'] / player_totals_basic_cleaned['attempted_two_point_field_goals']
player_totals_basic_pg['3P/G'] = player_totals_basic_cleaned['made_three_point_field_goals'] / games_played
player_totals_basic_pg['3PA/G'] = player_totals_basic_cleaned['attempted_three_point_field_goals'] / games_played
player_totals_basic_pg['3P%'] = player_totals_basic_cleaned['made_three_point_field_goals'] / player_totals_basic_cleaned['attempted_three_point_field_goals']
player_totals_basic_pg['FT/G'] = player_totals_basic_cleaned['made_free_throws'] / games_played
player_totals_basic_pg['FTA/G'] = player_totals_basic_cleaned['attempted_free_throws'] / games_played
player_totals_basic_pg['FT%'] = player_totals_basic_cleaned['made_free_throws'] / player_totals_basic_cleaned['attempted_free_throws']
player_totals_basic_pg['ORB/G'] = player_totals_basic_cleaned['offensive_rebounds'] / games_played
player_totals_basic_pg['DRB/G'] = player_totals_basic_cleaned['defensive_rebounds'] / games_played
player_totals_basic_pg['TRB/G'] = (player_totals_basic_cleaned['offensive_rebounds'] + player_totals_basic_cleaned['defensive_rebounds']) / games_played
player_totals_basic_pg['AST/G'] = player_totals_basic_cleaned['assists'] / games_played
player_totals_basic_pg['STL/G'] = player_totals_basic_cleaned['steals'] / games_played
player_totals_basic_pg['BLK/G'] = player_totals_basic_cleaned['blocks'] / games_played
player_totals_basic_pg['TOV/G'] = player_totals_basic_cleaned['turnovers'] / games_played
player_totals_basic_pg.to_csv('2000_2024_player_totals_basic.csv')
player_totals_basic_pg

MP/G       GS%       PS/G      FG/G  \
season_end name                                                        
2000       A.C. Green       23.524390  1.000000   5.036585  2.109756   
           A.J. Bramlett     7.625000  0.000000   1.000000  0.500000   
           Aaron McKie      23.804878  0.170732   7.963415  2.975610   
           Aaron Williams   19.074074  0.000000   7.604938  2.901235   
           Adam Keefe        9.741935  0.048387   2.177419  0.854839   
...                               ...       ...        ...       ...   
2024       Zach Collins     26.129032  0.741935  12.935484  4.935484   
           Zach LaVine      35.347826  0.913043  20.086957  6.956522   
           Zeke Nnaji        9.647059  0.000000   3.235294  1.176471   
           Ziaire Williams  18.918919  0.216216   8.027027  2.864865   
           Zion Williamson  30.375000  1.000000  21.968750  8.656250   

                                  FGA       FG%      2P/G      2PA/G  \
season_end name                                                        
2000       A.C. Green        4.719512  0.447028  2.097561   4.670732   
           A.J. Bramlett     2.625000  0.190476  0.500000   2.625000   
           Aaron McKie       7.231707  0.411467  2.439024   5.756098   
           Aaron Williams    5.555556  0.522222  2.901235   5.518519   
           Adam Keefe        2.096774  0.407692  0.854839   2.080645   
...                               ...       ...       ...        ...   
2024       Zach Collins      9.967742  0.495146  3.838710   6.451613   
           Zach LaVine      15.434783  0.450704  4.565217   8.478261   
           Zeke Nnaji        2.470588  0.476190  1.029412   2.029412   
           Ziaire Williams   7.054054  0.406130  1.675676   3.540541   
           Zion Williamson  14.812500  0.584388  8.562500  14.531250   

                                 2P%      3P/G  ...      FT/G     FTA/G  \
season_end name                                 ...                       
2000       A.C. Green       0.449086  0.012195  ...  0.804878  1.158537   
           A.J. Bramlett    0.190476  0.000000  ...  0.000000  0.000000   
           Aaron McKie      0.423729  0.536585  ...  1.475610  1.780488   
           Aaron Williams   0.525727  0.000000  ...  1.802469  2.481481   
           Adam Keefe       0.410853  0.000000  ...  0.467742  0.580645   
...                              ...       ...  ...       ...       ...   
2024       Zach Collins     0.595000  1.096774  ...  1.967742  2.483871   
           Zach LaVine      0.538462  2.391304  ...  3.782609  4.434783   
           Zeke Nnaji       0.507246  0.147059  ...  0.735294  1.147059   
           Ziaire Williams  0.473282  1.189189  ...  1.108108  1.297297   
           Zion Williamson  0.589247  0.093750  ...  4.562500  6.875000   

                                 FT%     ORB/G     DRB/G     TRB/G     AST/G  \
season_end name                                                                
2000       A.C. Green       0.694737  1.951220  3.975610  5.926829  0.975610   
           A.J. Bramlett         NaN  1.500000  1.250000  2.750000  0.000000   
           Aaron McKie      0.828767  0.573171  2.426829  3.000000  2.926829   
           Aaron Williams   0.726368  1.962963  3.086420  5.049383  0.716049   
           Adam Keefe       0.805556  0.725806  1.467742  2.193548  0.548387   
...                              ...       ...       ...       ...       ...   
2024       Zach Collins     0.792208  1.903226  4.064516  5.967742  3.290323   
           Zach LaVine      0.852941  0.347826  5.000000  5.347826  3.956522   
           Zeke Nnaji       0.641026  1.117647  1.058824  2.176471  0.500000   
           Ziaire Williams  0.854167  0.675676  2.810811  3.486486  1.351351   
           Zion Williamson  0.663636  1.875000  4.062500  5.937500  4.625000   

                               STL/G     BLK/G     TOV/G  
season_end name                                           
2000       A.C. Green   

In [18]:
player_totals_advanced = pd.read_csv('2000_2024_player_totals_advanced.csv', index_col=0)
player_totals_advanced

,slug,name,positions,age,team,games_played,minutes_played,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,...,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,is_combined_totals
season_end,,,,,,,,,,,,,,,,,,,,,
2000,abdulta01,Tariq Abdul-Wahad,[<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>],25,Team.ORLANDO_MAGIC,46,1205,14.4,0.484,0.041,...,23.3,0.4,1.6,2.0,0.082,-1.0,0.2,-0.8,0.4,False
2000,abdulta01,Tariq Abdul-Wahad,[<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>],25,Team.DENVER_NUGGETS,15,373,10.8,0.448,0.015,...,19.7,0.0,0.2,0.2,0.023,-1.7,-0.9,-2.5,0.0,False
2000,abdursh01,Shareef Abdur-Rahim,[<Position.SMALL_FORWARD: 'SMALL FORWARD'>],23,Team.VANCOUVER_GRIZZLIES,82,3223,20.2,0.547,0.075,...,25.0,6.2,2.6,8.8,0.132,2.6,-0.4,2.2,3.4,False
2000,alexaco01,Cory Alexander,[<Position.POINT_GUARD: 'POINT GUARD'>],26,Team.DENVER_NUGGETS,29,329,8.8,0.381,0.357,...,18.0,-0.5,0.4,-0.1,-0.012,-4.1,1.8,-2.4,0.0,False
2000,allenra02,Ray Allen,[<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>],24,Team.MILWAUKEE_BUCKS,82,3070,20.6,0.570,0.288,...,25.6,9.0,1.0,10.1,0.157,4.7,-1.1,3.6,4.3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,youngth01,Thaddeus Young,[<Position.POWER_FORWARD: 'POWER FORWARD'>],35,Team.TORONTO_RAPTORS,11,99,14.4,0.600,0.111,...,15.2,0.1,0.1,0.2,0.097,-1.9,1.1,-0.8,0.0,False
2024,youngtr01,Trae Young,[<Position.POINT_GUARD: 'POINT GUARD'>],25,Team.ATLANTA_HAWKS,36,1312,20.9,0.577,0.462,...,31.2,3.0,0.3,3.2,0.118,5.3,-2.3,3.0,1.6,False
2024,yurtsom01,Omer Yurtseven,[<Position.CENTER: 'CENTER'>],25,Team.UTAH_JAZZ,23,254,11.8,0.529,0.197,...,16.7,-0.1,0.3,0.2,0.033,-4.0,-0.8,-4.8,-0.2,False


In [19]:
player_totals_advanced_cleaned = player_totals_advanced.groupby('season_end')
agg_functions = {'slug': 'last', 'positions': 'last', 'age': 'last', 'team': 'last'}
agg_functions.update(dict(zip(player_totals_advanced.columns[5:], ['sum'] * len(player_totals_advanced.columns[5:]))))
player_totals_advanced_cleaned = player_totals_advanced_cleaned.transform(lambda x: x).groupby(['season_end', 'name']).aggregate(agg_functions)
player_totals_advanced_cleaned

slug  \
season_end name                         
2000       A.C. Green       greenac01   
           A.J. Bramlett    bramlaj01   
           Aaron McKie      mckieaa01   
           Aaron Williams   williaa01   
           Adam Keefe       keefead01   
...                               ...   
2024       Zach Collins     colliza01   
           Zach LaVine      lavinza01   
           Zeke Nnaji       nnajize01   
           Ziaire Williams  willizi02   
           Zion Williamson  willizi01   

                                                                positions  \
season_end name                                                             
2000       A.C. Green         [<Position.POWER_FORWARD: 'POWER FORWARD'>]   
           A.J. Bramlett                    [<Position.CENTER: 'CENTER'>]   
           Aaron McKie      [<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>]   
           Aaron Williams     [<Position.POWER_FORWARD: 'POWER FORWARD'>]   
           Adam Keefe                       [<Position.CENTER: 'CENTER'>]   
...                                                                   ...   
2024       Zach Collins                     [<Position.CENTER: 'CENTER'>]   
           Zach LaVine      [<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>]   
           Zeke Nnaji         [<Position.POWER_FORWARD: 'POWER FORWARD'>]   
           Ziaire Williams    [<Position.SMALL_FORWARD: 'SMALL FORWARD'>]   
           Zion Williamson    [<Position.POWER_FORWARD: 'POWER FORWARD'>]   

                            age                       team  games_played  \
season_end name                                                            
2000       A.C. Green        36    Team.LOS_ANGELES_LAKERS            82   
           A.J. Bramlett     23   Team.CLEVELAND_CAVALIERS             8   
           Aaron McKie       27    Team.PHILADELPHIA_76ERS            82   
           Aaron Williams    28    Team.WASHINGTON_WIZARDS            81   
           Adam Keefe        29             Team.UTAH_JAZZ            62   
...                         ...                        ...           ...   
2024       Zach Collins      26     Team.SAN_ANTONIO_SPURS            31   
           Zach LaVine       28         Team.CHICAGO_BULLS            23   
           Zeke Nnaji        23        Team.DENVER_NUGGETS            34   
           Ziaire Williams   22     Team.MEMPHIS_GRIZZLIES            37   
           Zion Williamson   23  Team.NEW_ORLEANS_PELICANS            32   

                            minutes_played  player_efficiency_rating  \
season_end name                                                        
2000       A.C. Green                 1929                      11.2   
           A.J. Bramlett                61                      -0.4   
           Aaron McKie                1952                      13.2   
           Aaron Williams             1545                      17.7   
           Adam Keefe                  604                       7.6   
...                                    ...                       ...   
2024       Zach Collins                810                      14.1   
           Zach LaVine                 813                      15.8   
           Zeke Nnaji                  328                      11.7   
           Ziaire Williams             700                       9.9   
           Zion Williamson             972                      22.3   

                            true_shooting_percentage  \
season_end name                                        
2000       A.C. Green                          0.482   
           A.J. Bramlett                       0.190   
           Aaron McKie                         0.497   
           Aaron Williams                      0.572   
           Adam Keefe                          0.463   
...                                              ...   
2024       Zach Collins                        0.585   
           Zach LaVine                         0.578   
           

In [42]:
player_totals_advanced_pg = pd.DataFrame()
games_played = player_totals_basic_cleaned['games_played']
player_totals_advanced_pg['PER'] = player_totals_advanced_cleaned['player_efficiency_rating']
player_totals_advanced_pg['TS%'] = player_totals_advanced_cleaned['true_shooting_percentage']
player_totals_advanced_pg['eFG%'] = (player_totals_basic_cleaned['made_two_point_field_goals'] + 1.5 * player_totals_basic_cleaned['made_three_point_field_goals']) / player_totals_basic_cleaned['attempted_field_goals']
player_totals_advanced_pg['3PAr'] = player_totals_basic_cleaned['attempted_three_point_field_goals'] / player_totals_basic_cleaned['attempted_field_goals']
player_totals_advanced_pg['FTr'] = player_totals_basic_cleaned['attempted_free_throws'] / player_totals_basic_cleaned['attempted_field_goals']
player_totals_advanced_pg['ORB%'] = player_totals_advanced_cleaned['offensive_rebound_percentage']
player_totals_advanced_pg['DRB%'] = player_totals_advanced_cleaned['defensive_rebound_percentage']
player_totals_advanced_pg['TRB%'] = player_totals_advanced_cleaned['total_rebound_percentage']
player_totals_advanced_pg['AST%'] = player_totals_advanced_cleaned['assist_percentage']
player_totals_advanced_pg['USG%'] = player_totals_advanced_cleaned['usage_percentage']
player_totals_advanced_pg['STL%'] = player_totals_advanced_cleaned['steal_percentage']
player_totals_advanced_pg['BLK%'] = player_totals_advanced_cleaned['block_percentage']
player_totals_advanced_pg['TOV%'] = player_totals_advanced_cleaned['turnover_percentage']
player_totals_advanced_pg.to_csv('2000_2024_player_totals_advanced.csv')

In [21]:
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players

def get_shot_data(player_full_name: str, season: str):
    player_dictionary = players.get_players()
    try:
        player_info = [player for player in player_dictionary if player['full_name'] == player_full_name][0]
    except Exception as e:
        return None
    player_id = player_info['id']

    player_shotlog = shotchartdetail.ShotChartDetail(team_id = 0, player_id = player_id, season_nullable=season, context_measure_simple = 'FGA', season_type_all_star = ['Regular Season'])

    player_df = player_shotlog.get_data_frames()[0]
    return player_df

In [29]:
shot_data_cleaned = pd.DataFrame()
shot_data = get_shot_data('LeBron James', season='2009-10')
shot_data

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0020900001,5,2544,LeBron James,1610612739,Cleveland Cavaliers,1,11,3,...,Right Side Center(RC),16-24 ft.,23,129,194,1,1,20091027,CLE,BOS
1,Shot Chart Detail,0020900001,9,2544,LeBron James,1610612739,Cleveland Cavaliers,1,10,6,...,Center(C),Less Than 8 ft.,1,-2,12,1,1,20091027,CLE,BOS
2,Shot Chart Detail,0020900001,47,2544,LeBron James,1610612739,Cleveland Cavaliers,1,6,28,...,Right Side Center(RC),24+ ft.,24,118,217,1,0,20091027,CLE,BOS
3,Shot Chart Detail,0020900001,58,2544,LeBron James,1610612739,Cleveland Cavaliers,1,4,47,...,Left Side Center(LC),24+ ft.,33,-172,287,1,1,20091027,CLE,BOS
4,Shot Chart Detail,0020900001,60,2544,LeBron James,1610612739,Cleveland Cavaliers,1,4,21,...,Center(C),Less Than 8 ft.,0,1,9,1,1,20091027,CLE,BOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1523,Shot Chart Detail,0020901150,334,2544,LeBron James,1610612739,Cleveland Cavaliers,3,0,0,...,Left Side Center(LC),16-24 ft.,22,-109,198,1,0,20100406,CLE,TOR
1524,Shot Chart Detail,0020901150,393,2544,LeBron James,1610612739,Cleveland Cavaliers,4,6,42,...,Center(C),Less Than 8 ft.,0,7,7,1,1,20100406,CLE,TOR
1525,Shot Chart Detail,0020901150,405,2544,LeBron James,1610612739,Cleveland Cavaliers,4,5,45,...,Right Side Center(RC),24+ ft.,26,108,241,1,0,20100406,CLE,TOR
1526,Shot Chart Detail,0020901150,408,2544,LeBron James,1610612739,Cleveland Cavaliers,4,5,22,...,Right Side Center(RC),16-24 ft.,21,99,186,1,1,20100406,CLE,TOR


In [23]:
shot_data_aggregated = pd.DataFrame()
shot_data_aggregated['shot_made'] = shot_data.groupby('SHOT_ZONE_BASIC')['SHOT_MADE_FLAG'].sum()
shot_data_aggregated['shots_attempted'] = shot_data.groupby('SHOT_ZONE_BASIC')['SHOT_ATTEMPTED_FLAG'].sum()
shot_data_aggregated['shots_pct'] = shot_data_aggregated['shot_made'] / shot_data_aggregated['shots_attempted']
shot_data_aggregated = shot_data_aggregated.unstack().reorder_levels([1, 0]).sort_index()

In [24]:
player_totals_advanced_pg.groupby('season_end').get_group(2000).index.get_level_values('name').unique()
# player_totals_advanced_pg

Index(['A.C. Green', 'A.J. Bramlett', 'Aaron McKie', 'Aaron Williams',
       'Adam Keefe', 'Adonal Foyle', 'Adrian Griffin', 'Al Harrington',
       'Alan Henderson', 'Aleksandar Radojević',
       ...
       'Vonteego Cummings', 'Voshon Lenard', 'Wally Szczerbiak',
       'Walt Williams', 'Walter McCarty', 'Wayne Turner', 'Wesley Person',
       'Will Perdue', 'William Avery', 'Žan Tabak'],
      dtype='object', name='name', length=439)

In [37]:
shot_data_agg = pd.DataFrame()
for i in tqdm(range(17, 25)):
    season_start = f'200{i}' if i < 10 else f'20{i}'
    season_end = f'0{i+1}' if i+1 < 10 else f'{i+1}'
    season_end_idx = f'20{season_end}'
    season = f'{season_start}-{season_end}'
    for player in player_totals_advanced_pg.groupby('season_end').get_group(int(season_end_idx)).index.get_level_values('name').unique():
        failed = 0
        while(True):
            try:
                player_shot_data = get_shot_data(player, season)
                if player_shot_data is None or len(player_shot_data) == 0:
                    print(f'Player data not found: {player}')
                    break
                player_shot_data_agg = pd.DataFrame()
                player_shot_data_agg['shot_made'] = player_shot_data.groupby('SHOT_ZONE_BASIC')['SHOT_MADE_FLAG'].sum()
                player_shot_data_agg['shots_attempted'] = player_shot_data.groupby('SHOT_ZONE_BASIC')['SHOT_ATTEMPTED_FLAG'].sum()
                player_shot_data_agg['shots_pct'] = player_shot_data_agg['shot_made'] / player_shot_data_agg['shots_attempted']
                player_shot_data_series = player_shot_data_agg.unstack().reorder_levels([1, 0]).sort_index()
                player_shot_data_agg = pd.DataFrame()
                player_shot_data_agg['METRICS'] = player_shot_data_series
                player_shot_data_agg['season_end'] = len(player_shot_data_agg) * [f'200{season_end}' if int(season_end) < 10 else f'20{season_end}']
                player_shot_data_agg['player'] = len(player_shot_data_agg) * [player]
                player_shot_data_agg.set_index(['player', 'season_end'], inplace=True, append=True)
                shot_data_agg = pd.concat([shot_data_agg, player_shot_data_agg])
                shot_data_agg.to_csv('2018_2024_player_shot_metrics.csv')
            except Exception as e:
                failed+=1
                print(f'Failed {failed} {"times" if failed > 1 else "time"}. Waiting 5 minutes...')
                print(f'Error displayed: {e}')
                print('5 minute progress...')
                for i in tqdm(range(5 * 60)):
                    time.sleep(1)
                continue
            break

  0%|          | 0/8 [00:00<?, ?it/s]

Failed 1 time. Waiting 5 minutes...
Error displayed: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
5 minute progress...


  0%|          | 0/300 [00:00<?, ?it/s]

Player data not found: Andrew White
Player data not found: Ante Žižić
Player data not found: Ben Moore
Player data not found: Boban Marjanović
Player data not found: Bogdan Bogdanović
Player data not found: Bojan Bogdanović
Player data not found: C.J. Miles
Player data not found: Cristiano Felício
Player data not found: Dario Šarić
Failed 1 time. Waiting 5 minutes...
Error displayed: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
5 minute progress...


  0%|          | 0/300 [00:00<?, ?it/s]

Failed 2 times. Waiting 5 minutes...
Error displayed: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
5 minute progress...


  0%|          | 0/300 [00:00<?, ?it/s]

Player data not found: Dennis Schröder
Player data not found: Derrick Walton
Player data not found: Dāvis Bertāns
Player data not found: Ersan İlyasova
Player data not found: Goran Dragić
Player data not found: J.J. Redick
Player data not found: J.R. Smith
Player data not found: Johnny O'Bryant
Player data not found: Jonas Valančiūnas
Player data not found: José Calderón
Player data not found: Juancho Hernangómez
Player data not found: Jusuf Nurkić
Player data not found: Kristaps Porziņģis
Player data not found: Manu Ginóbili
Player data not found: Marcus Morris
Player data not found: Matt Williams
Player data not found: Mike James
Player data not found: Miloš Teodosić
Player data not found: Mirza Teletović
Player data not found: Nenê
Player data not found: Nicolás Brussino
Player data not found: Nikola Jokić
Player data not found: Nikola Mirotić
Player data not found: Nikola Vučević
Player data not found: R.J. Hunter
Player data not found: Reggie Bullock
Player data not found: Skal La

KeyError: 2025